In [13]:
## Load essential packages ##

%reset -f

import os
import openai
import pandas as pd

In [14]:
## Load API Key from system memory

openai.api_key = os.getenv("OPENAI_API_KEY")

In [15]:
## Define the System Role 

def ConstructSystemRole():

    SystemRole = ''' Your role is to help create a descriptive category title in english for a group of responses in french collected from mothers that have not vaccinated
their children.  You may also be given responses from two other groups of responses so that you can compare the responses and create an 
accurate description.  You also need to say whether the responses in the cluster are fairly consistent or quite divergent in topics.

Here are a couple of examples of the format of the output in English:

Assistant:  Fear of COVID-19 vaccine side effects, Consistent
Assistant:  Vague responses, Divergent

'''
      
    return SystemRole


In [16]:
# Different models available via openai:  gpt-3.5-turbo, gpt-4, gpt-4-0314

SystemRole = ConstructSystemRole()

def request_completion(prompt,SystemRole):

    response = openai.ChatCompletion.create(
    model='gpt-4-0125-preview',
        messages = [
            {"role": "system", "content": SystemRole},
            {"role": "user", "content":prompt}
        ],
    temperature=0,
    max_tokens=2000,
    top_p=0.5,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

    #print(response['choices'][0]['message']['content'])
    return response['choices'][0]['message']['content']

In [ ]:
# Look at the first 10 rows of the data in cluster 1

Cluster1 = pd.read_excel('largeEmbeddingOutputsAllDataRed250_50Clusters/textOutput1.xlsx',engine='openpyxl')

subset_long_string = Cluster1[0].sample(n=10).str.cat(sep='\n')
subset_long_string
print(len(Cluster1))
#print(str(Cluster1.iloc[0,0]))

In [18]:
# This is not a super well written cell at this point, but the goal was to load responses in each cluster and neighboring clusters to compare the responses and create a descriptive title for the cluster.  

import ast

clusters = 75
response = ''

for index in range(1,clusters): 

    if index == 1:
        
        Cluster1 = pd.read_excel('largeEmbeddingOutputsAllDataRed250_75Clusters/textOutput1.xlsx',engine='openpyxl')
        Cluster2 = pd.read_excel('largeEmbeddingOutputsAllDataRed250_75Clusters/textOutput2.xlsx',engine='openpyxl')

        if len(Cluster2) < 15:
            samp = len(Cluster2)
        else:
            samp = 15
        
        subset = Cluster2[0].sample(n=samp).str.cat(sep='\n')
        complete = Cluster1[0].str.cat(sep='\n')
        
        prompt = 'The following are responses for comparison: \n '+subset+ '\n The following are the actual responses for you to create a descriptive title:'+complete
        
    elif index == clusters:
        
        Cluster1 = pd.read_excel('largeEmbeddingOutputsAllDataRed250_75Clusters/textOutput100.xlsx',engine='openpyxl')
        Cluster2 = pd.read_excel('largeEmbeddingOutputsAllDataRed250_75Clusters/textOutput99.xlsx',engine='openpyxl')

        if len(Cluster2) < 15:
            samp = len(Cluster2)
        else:
            samp = 15
        
        subset = Cluster2[0].sample(n=samp).str.cat(sep='\n')
        complete = Cluster1[0].str.cat(sep='\n')
        
        prompt = 'The following are responses for comparison: \n '+subset+'\n The following are the actual responses for you to create a descriptive title:'+complete
      
    else:
        Cluster1 = pd.read_excel('largeEmbeddingOutputsAllDataRed250_75Clusters/textOutput'+str(index)+'.xlsx',engine='openpyxl')
        Cluster2 = pd.read_excel('largeEmbeddingOutputsAllDataRed250_75Clusters/textOutput'+str(index-1)+'.xlsx',engine='openpyxl')
        Cluster3 = pd.read_excel('largeEmbeddingOutputsAllDataRed250_75Clusters/textOutput'+str(index+1)+'.xlsx',engine='openpyxl')

        if len(Cluster2) < 15:
            samp = len(Cluster2)
        else:
            samp = 15

        if len(Cluster3) < 15:
            samp2 = len(Cluster2)
        else:
            samp2 = 15
        
        subset = Cluster2[0].sample(n=samp).str.cat(sep='\n')
        subset2 = Cluster3[0].sample(n=samp2).str.cat(sep='\n')
        complete = Cluster1[0].str.cat(sep='\n')
        
        prompt = 'The following are responses for comparison in one subset: \n '+subset+ '\n The following are responses for another subset:\n'+subset2+'\n The following are the actual responses for you to create a descriptive title:'+complete

    
    assistantResponse = request_completion(prompt,SystemRole)
    print(assistantResponse)

    response = response+str(assistantResponse)+'\n'
        
print(response)

Awaiting 9-Month Vaccination Milestone, Consistent
Awaiting Age for Completion of Vaccination Schedule, Consistent
Vaccination Schedule in Progress, Consistent
Ongoing Vaccination Process, Consistent
Barriers to Vaccination: Accessibility, Availability, and Scheduling Issues, Consistent
Barriers to Complete Vaccination: Misinformation, Accessibility, and Systemic Issues, Divergent
Minimum Child Quota Not Met for Vaccine Administration, Consistent
Concerns and Preferences Regarding Injectable Vaccines, Divergent
Vaccination Completed According to Parents, but Documentation Issues Present, Consistent
Barriers to Childhood Vaccination: Logistical, Financial, and Health Concerns, Divergent
Concerns about Vaccine Side Effects and Financial Burden of Post-Vaccination Care, Consistent
Refus de vaccination pour des raisons variées: conflits personnels, décisions parentales, et interdictions religieuses, Consistent
Barriers to Child Vaccination: Parental Absence, Negligence, and Misinformation,